In [1]:
import os
import numpy as np
import spectral.io.envi as envi
import matplotlib.pyplot as plt
from hsi_utils import *

from sklearn.decomposition import PCA

In [2]:
#if using a jupyter notbook need a graphical backend TkaAgg or Qt5Agg if using spyder type %matplotlib auto in console
import matplotlib
matplotlib.use('TkAgg') 

In [35]:
# Define the path to the main data folder: code will iterate trough relvant files
main_data_folder = './data/img_test/ref_corrected'    

#check if data path exists !
print(os.path.isdir(main_data_folder))

# Initialize the HSI dataset and define file extension: contains all paths of hdr and data files
dataset =HsiDataset(main_data_folder,data_ext='ref') #raw files from the camera have '.hyspex' extension , corrected files have '.ref' extension
nb_images = len(dataset)
print(f"dataset contains {nb_images} image(s)")

True
dataset contains 1 image(s)


In [36]:
HSIreader = HsiReader(dataset)

In [37]:
# Loop through each hyperspectral image in the dataset
# for idx in range(len(dataset)):
#     HSIreader.read_image(idx)

#choose an image to process e.g. first img idx=0
idx=0   
HSIreader.read_image(idx)
print(f"read image{HSIreader.current_name}")

metadata = HSIreader.current_metadata
print(metadata)

#define wavelenrghts (for plots mostly)
wv =HSIreader.get_wavelength()
wv = [int(l) for l in wv]

# Get the pseudo rgb imahe from hyperspectral data and plot image
pseudo_rgb= HSIreader.get_rgb()
# plt.figure()
# plt.imshow(pseudo_rgb)
# plt.axis('off')
# plt.show(block=False)

read imageBarley_swir_30cm_SWIR_384me_SN3145_2700us_2024-07-19T100942_raw_ref
{'description': 'Frameperiod = 3500\nIntegration time = 2700\nBinning = 1\nNumber of frames = 1094\nAperture size = 0.005000\ndw = 1\nEQ = 0\nLens = 1\nCamera configuration (lens/filter/gain) = 30 cm\nHSNR = 0\nCalibAvailable = 0\nNumber of background = 200\nPixelsize x = 0.00072999999999999996\nPixelsize y = 0.00072999999999999996\nID = SWIR_384me_SN3145\nComment =\nSerialnumber = 3145\nScanningmode = Ground', 'samples': '384', 'lines': '1094', 'bands': '288', 'header offset': '0', 'file type': 'ENVI Standard', 'data type': '12', 'interleave': 'bip', 'byte order': '0', 'data ignore value': '65535', 'default bands': ['50', '130', '220'], 'wavelength units': 'nm', 'wavelength': ['953.320143', '958.762294', '964.204445', '969.646597', '975.088748', '980.530899', '985.973051', '991.415202', '996.857353', '1002.299505', '1007.741656', '1013.183807', '1018.625959', '1024.068110', '1029.510261', '1034.952413', '104

In [38]:
#check spectral data by manually selecting pixel-spectrum samples and plot
HSIreader.get_spectrum();

RGB channels: [50, 130, 220]


In [17]:
# Extract spectral sample from HSI for analysis

#read HSI specs without laoding it in memory!
hsi=HSIreader.current_image
n_rows, n_cols, n_channels =hsi.shape

n_samples =1000
# draw random samples and load them as a data array
x_idx = np.random.randint(0, n_cols, size=n_samples)
y_idx = np.random.randint(0, n_rows, size=n_samples)
spectral_samples = np.zeros((n_samples, n_channels), dtype=hsi.dtype)
coords = list(zip(y_idx, x_idx))
spectral_samples = np.array(HSIreader.extract_pixels(coords))

In [18]:
plt.figure()
for i in range(n_samples):
    plt.plot(wv, spectral_samples[i, :], label=f'Sample {i+1}' if i < 5 else "", alpha=0.6)
plt.xlabel("Wavelength")
plt.ylabel("Absorbance")
plt.title("Spectral samples")
plt.show(block=False)


In [20]:
#perform PCA
#In PCA space data X= TP' i.e. factorial decomposition into loadings (vectorial subspaces) and scores (orthogonal distances to loadings)

nb_pca_comp =3
pca = PCA(n_components=nb_pca_comp)
pca_scores = pca.fit_transform(spectral_samples)
pca_loadings =pca.components_.T*np.sqrt(pca.explained_variance_) 
      
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure()
for i in range(np.shape(pca_loadings)[1]):
    lab= 'PC'+str(i+1)
    plt.plot(wv,pca_loadings[:,i],default_colors[i], label=lab)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Absorbance")  
plt.grid()
plt.title('Principal components')  
plt.show(block=False)

#Or plot one by one to avoid Variance scaling
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i in range(np.shape(pca_loadings)[1]):
    plt.figure()
    plt.plot(wv,pca_loadings[:,i],default_colors[i])
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Absorbance")  
    lab= 'PC'+str(i+1)
    plt.title(lab) 
    plt.grid()  
plt.show()

In [21]:
# Plot the score distribution according to PCi / PCj
plt.figure()
plt.scatter(pca_scores[:, 0], pca_scores[:, 1], alpha=0.6, c='blue', edgecolor='k', s=50)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Score Plot PC1/PC2")
plt.grid()
plt.show(block=False)

In [22]:
plt.figure()
plt.scatter(pca_scores[:, 1], pca_scores[:, 2], alpha=0.6, c='blue', edgecolor='k', s=50)
plt.xlabel("PC2")
plt.ylabel("PC3")
plt.title("Score Plot PC2/PC3")
plt.grid()
plt.show(block=False)

In [23]:
#project HSI on PCA loadings -> obtain global scores (If n_samples too small, resample)

score_img = HSIreader.project_pca_scores(pca_loadings)
for s in range(pca_loadings.shape[1]):
        plt.figure()
        plt.imshow(score_img[:,:,s])
        plt.title(f'Score image PC{s+1}')
        plt.axis('off')
        plt.show(block=False)